In [1]:
import sys
import os

try:
    if changed:
        raise Exception("changed")
except NameError:
    changed = False
    sys.path.append("../")
    os.chdir("../")

The goal of this notebook is to show the main parts for how to use the source code for the project Unrolled Optimization Deep Learning for Sparse Signals Restoration with a few results.
The experiments that are presented in the report can all be reproduced with the code in this notebook, by changing the configs files used to run the commands.

## 1 - Generate the data

WARNING: The datasets are extremely heavy and can take up a lot of hard drive space (around 100Gb for both MassBank and synthetic data). This is due to the sizes of the kernel matrix $H$ which is stored for every signal.

### MassBank Data

### Variable Ricker Kernel

In [ ]:
!python ./data/generate_data.py --Dataset massbank

### Fixed Ricker kernel

This dataset is used when running the experiments that allow to learn the kernel $H$ with the neural networks.

In [ ]:
!python ./data/generate_data.py --Dataset massbank_fixed --Fixed

## Synthetic Data

The synthetic data is generated using the notebook `gen_sparse_signal_draft.ipynb`, where the parameters are adapted to the experiments to run.

### Synthetic easy dataset

This dataset is called easy because there is only one sparsity level that is lower and the noise level is low as well as indicated in the report. However, the kernels are still different inside the dataset and vary in standard deviation.

In [9]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_easy" --sparsities 0.01 --num_samples 100 --noise_level 0.5 --kernel_type gaussian identity gaussian_heteroscedastic sparse_random

['gaussian', 'identity']
[0.1, 0.2]
[0.1, 0.2]
8
0.1 0.1 gaussian
0 th out of 8
0.1 0.1 identity
10 th out of 8
0.1 0.2 gaussian
20 th out of 8
0.1 0.2 identity
30 th out of 8
0.2 0.1 gaussian
40 th out of 8
0.2 0.1 identity
50 th out of 8
0.2 0.2 gaussian
60 th out of 8
0.2 0.2 identity
70 th out of 8


### Synthetic hard dataset

This dataset is called hard because we have a higher sparsity level and a higher noise level, and they are both not fixed. The kernels are the same.

In [2]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_hard" --sparsities 0.01 0.05 --num_samples 20 --noise_level 0.5 1 2 --kernel_type gaussian identity gaussian_heteroscedastic sparse_random

Kernel types:
['gaussian', 'identity', 'gaussian_heteroscedastic', 'sparse_random']
Noise levels:
[0.5, 1.0, 2.0]
Sparsities:
[0.01, 0.05]
24
0.01 0.5 gaussian
0 th out of 24
0.01 0.5 identity
20 th out of 24
0.01 0.5 gaussian_heteroscedastic
40 th out of 24
0.01 0.5 sparse_random
60 th out of 24
0.01 1.0 gaussian
80 th out of 24
0.01 1.0 identity
100 th out of 24
0.01 1.0 gaussian_heteroscedastic
120 th out of 24
0.01 1.0 sparse_random
140 th out of 24
0.01 2.0 gaussian
160 th out of 24
0.01 2.0 identity
180 th out of 24
0.01 2.0 gaussian_heteroscedastic
200 th out of 24
0.01 2.0 sparse_random
220 th out of 24
0.05 0.5 gaussian
240 th out of 24
0.05 0.5 identity
260 th out of 24
0.05 0.5 gaussian_heteroscedastic
280 th out of 24
0.05 0.5 sparse_random
300 th out of 24
^C
Traceback (most recent call last):
  File "/Users/aliramlaoui/Library/CloudStorage/GoogleDrive-ramlaouiali@gmail.com/My Drive/Cours 3A/AST/project/unrolled-nn-restoration/./data/generate_synthetic_data.py", line 245, 

### Fixed Kernel type synthetic dataset

These datasets are used for running experiments with single kernel type and compare the performances of the models with different kernel types.

In [4]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_identity" --sparsities 0.01 --num_samples 200 --noise_level 0.2 0.5 1 --kernel_type identity

Kernel types:
['identity']
Noise levels:
[0.2, 0.5, 1.0]
Sparsities:
[0.01]
3
0.01 0.2 identity
0 th out of 3
0.01 0.5 identity
10 th out of 3
0.01 1.0 identity
20 th out of 3
^C
Traceback (most recent call last):
  File "/Users/aliramlaoui/Library/CloudStorage/GoogleDrive-ramlaouiali@gmail.com/My Drive/Cours 3A/AST/project/unrolled-nn-restoration/./data/generate_synthetic_data.py", line 245, in <module>
    np.save(
  File "/Users/aliramlaoui/Library/CloudStorage/GoogleDrive-ramlaouiali@gmail.com/My Drive/Cours 3A/AST/project/unrolled-nn-restoration/venv/lib/python3.10/site-packages/numpy/lib/npyio.py", line 546, in save
    format.write_array(fid, arr, allow_pickle=allow_pickle,
  File "/Users/aliramlaoui/Library/CloudStorage/GoogleDrive-ramlaouiali@gmail.com/My Drive/Cours 3A/AST/project/unrolled-nn-restoration/venv/lib/python3.10/site-packages/numpy/lib/format.py", line 730, in write_array
    array.tofile(fp)
KeyboardInterrupt


In [ ]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_gaussian" --sparsities 0.01 --num_samples 200 --noise_level 0.2 0.5 1 --kernel_type gaussian

In [ ]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_gaussian_heteroscedastic" --sparsities 0.01 --num_samples 200 --noise_level 0.2 0.5 1 --kernel_type gaussian_heteroscedastic

In [ ]:
!python ./data/generate_synthetic_data.py --dataset_name "synthetic_sparse_random" --sparsities 0.01 --num_samples 200 --noise_level 0.2 0.5 1 --kernel_type sparse_random

## 2 - Train a model

It is possible to just train a model with the command line and check the results with Weights and Biases. In the Jupyter notebook, you need to be able to input whether you want to log the training metrics on wandb or not if you can interact with the command line if --is_debug is not set. Otherwise, it is just possible to run directly the line on the command line.

The parameters used as flags here are optional and will just overwrite the parameters in the config file. The config file is the one that is used by default if no config file is specified.

In [ ]:
# Primal Dual model
!python -m src.main --config primal_dual --n_epochs 1 --data_path ./data/massbank --batch_size 64 --lr 0.0001 --is_debug

In [ ]:
# ISTA model
!python -m src.main --config ista --n_epochs 1 --data_path ./data/massbank --batch_size 64 --lr 0.0001 --is_debug

## 3 - Run an experiment

Experiments are run with the command line and the ones in the report are predefined in the `configs/experiments` folder. After choosing an experiment to run, the command line is the following:

In [ ]:
!python -m src.experiments --config learn_h_default_fixed_ista --is_debug

This command will run the experiment with the parameters in the config file, log the results in Weights and Biases if --is_debug is not given and save the model in the folder `models` with the name of the experiment.

It is then possible to use a trained model and only test it on the test sets that are specified in the corresponding config file. The command line is the following:

In [ ]:
!python -m src.experiments --config learn_h_default_fixed_ista --test --is_debug

The source code is entirely available in this repository and in order to get a more in depth understanding in how the models are implemented, you can check the `src` folder.